In [ ]:
# !pip install simpletransformers
import pandas as pd
import numpy as np
from simpletransformers.ner import NERModel,NERArgs
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
import torch
model = torch.load('../pkl_model/model_2000_true_multiprocessed_decoding_true_cuda')

#code to clean new summaries text
import nltk
import re
from bs4 import BeautifulSoup

# nltk.download("punkt")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    # text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    # text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text


In [ ]:

# data=pd.read_csv("../dataset/production_tags_v2.csv") #GIVE DATA PATH
data=pd.read_csv("../dataset/production_tags_JD_mergedkeys.csv") #GIVE DATA PATH

col_name = "job_desc" #ENTER COL NAME

data[col_name]=data[col_name].astype(str)
data[col_name]=data[col_name].apply(clean_text)

# data.sort_values("lead_id", inplace = True)
# data.drop_duplicates(subset ="summaries",
#                      keep = False, inplace = True)
# data.reset_index(inplace = True)
# print(data[col_name].str.len().sum())
data[col_name] = data[col_name].replace(',', ' ', regex=True)
data[col_name] = data[col_name].replace('\*', ' ', regex=True)
data[col_name] = data[col_name].replace('\s+', ' ', regex=True)

In [ ]:
print(len(data))

In [ ]:
data[col_name]

In [ ]:
data.reset_index(inplace = True)
def get_sen(sentence):
    pred_data,model_output=model.predict([sentence])
    df=pd.DataFrame(pred_data)
    df=df.transpose()
    df.columns=['AA']
    df['AA']=df['AA'].astype(str)
    df['AA']=df['AA'].str[2:-2]
    df[['Word', 'Pred']] = df['AA'].str.split("': '", 1, expand=True)
    df=df.drop(columns='AA')
    # if(len(df.loc[(df['Pred']=='B-ORG') | (df['Pred']=='I-ORG')])>0):
    #   print(df.loc[(df['Pred']=='B-ORG') | (df['Pred']=='I-ORG')])
    return df


def give_dataframe_return_list_of_TAGS(temp_data):
    ccc=0
    tagg_per_sent = []
    prev_tag="O"
    string=""
    for i in range(0,len(temp_data)):
        value=temp_data['Pred'][i]
        key=temp_data['Word'][i]
        if (value=="B-ORG"):
            if (len(string)>0):
                tagg_per_sent.append(string)
            string=key
            prev_tag="B-ORG"

        elif (value=="I-ORG"):
            string=string+" "+key
            prev_tag="I-ORG"
            ccc+=1
        else:
            if(len(string)>0):
                tagg_per_sent.append(string)
            string=""
            prev_tag="O"
        if(len(string)>0):
            tagg_per_sent.append(string)
        # if(ccc>=1):
        #   print("i-org-count",ccc)
        # print(tagg_per_sent)
    return tagg_per_sent

time_list=[]
import time
final_list=[]
for i in tqdm(range(0,len(data))):
    start_time = time.time() # starting of code block
    print("****",i,"****")
    length_shouldbe = len(data[col_name][i]) #no of characters
    print(length_shouldbe)
    temp_data=get_sen(data[col_name][i]) 
    len_till_now = temp_data['Word'].str.len().sum()+1+len(temp_data)
    #   print(len_till_now)
    while (len_till_now<length_shouldbe):
        try:
          aaa = get_sen(data[col_name][i][len_till_now:])
          kkk = temp_data.append(aaa)
          # print(aaa)
          temp_data = kkk.reset_index(drop=True)
          len_till_now = temp_data['Word'].str.len().sum()+1+len(temp_data)
        #       print(len_till_now)
        except:
          break
    end = time.time()-start_time # where search ends
    time_list.append(end)
    final_list.append(list(set(give_dataframe_return_list_of_TAGS(temp_data))))

In [ ]:
data['BERT_Tags']=final_list
data['Time']=time_list
# data.to_csv("BERT_prod_tagsv2.csv") #enter your output data name
data.to_excel("BERT_production_tags_JD_mergedkeys.xlsx")

In [ ]:
len(final_list)

In [ ]:
sum(time_list)

In [ ]:
# pip install  openpyxl

In [ ]:
data.loc[13,:]